# Começando com RAG
Fazer o LLM utilizar um arquivo selecionado, para aplicar RAG com langchain

### O que são embeddings?
Textos serão convertidos em vetores mas preservando o significado do conteúdo. A semelhança entre vetores é calculada utilizando o cosseno do ângulo entre eles (cosseno de singularidade), assim , dois vetores próximos terão ângulos pequenos e indicando alta similiaridade semântica.

## Importações e criação do modelo

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_openai import ChatOpenAI #, OpenAIEmbeddings pago
from langchain_community.embeddings import HuggingFaceEmbeddings # Gratis
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("API_KEY")
if API_KEY is None:
    raise ValueError("A chave da API não foi definida no .env")

modelo = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",     
    model_name="meta-llama/llama-3-8b-instruct"
)

## RAG em arquivo TXT

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Leitura do documento
documento = TextLoader(
    "../documents/GTB_gold_Nov23.txt",
    encoding="utf-8"
).load()

# Quebrando o texto em pedaços (split)
pedacos = RecursiveCharacterTextSplitter(
    chunk_size=1000, # Split a cada 1000 char
    chunk_overlap=100 # Sobreposição da chunksize para não tirar do contexto
).split_documents(documento)

# Utilizando o FAISS, uma base de dados vetorial simples e de uso gratuito
dados_recuperados = FAISS.from_documents(
    pedacos, embeddings
).as_retriever(search_kwargs={"k":2}) # Recuperar embedding com posição de semelhança como top 2

prompt_consulta_seguro = ChatPromptTemplate.from_messages(
    [
        ("system", "Responda usando exclusivamente o conteúdo fornecido"),
        ("human", "{query}\n\nContexto: \n{contexto}\n\nResposta: ")
    ]
)

cadeia = prompt_consulta_seguro | modelo | StrOutputParser()

def responder(pergunta:str):
    trechos = dados_recuperados.invoke(pergunta)
    contexto = "\n\n".join(um_trecho.page_content for um_trecho in trechos)
    return cadeia.invoke({"query": pergunta, "contexto": contexto})

print(responder("Como devo proceder caso tenha um item roubado?"))

/tmp/ipykernel_11672/3181071964.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/home/klovynho/projects/langchain-rag-pdf/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Se você tiver um item roubado, você deve entrar em contato com o Serviço Global Mastercard e realizar os passos de registro da ocorrência. Seguindo as orientações fornecidas no Guia de Benefícios para o programam de Cartão da Edição Mastercard Gold.


## RAG em um arquivo PDF

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Caminho dos arquivos
arquivos = [
    "../documents/GTB_standard_Nov23.pdf",
    "../documents/GTB_gold_Nov23.pdf",
    "../documents/GTB_platinum_Nov23.pdf"
]

# Concatenando todos os documentos
documentos = sum(
    [PyPDFLoader(arquivo).load() for arquivo in arquivos], []
)

# Quebrando o texto em pedaços (split)
pedacos = RecursiveCharacterTextSplitter(
    chunk_size=1000, # Split a cada 1000 char
    chunk_overlap=100 # Sobreposição da chunksize para não tirar do contexto
).split_documents(documentos)

# Utilizando o FAISS, uma base de dados vetorial simples e de uso gratuito
dados_recuperados = FAISS.from_documents(
    pedacos, embeddings
).as_retriever(search_kwargs={"k":2}) # Recuperar embedding com posição de semelhança como top 2

prompt_consulta_seguro = ChatPromptTemplate.from_messages(
    [
        ("system", "Responda usando exclusivamente o conteúdo fornecido"),
        ("human", "{query}\n\nContexto: \n{contexto}\n\nResposta: ")
    ]
)

cadeia = prompt_consulta_seguro | modelo | StrOutputParser()

def responder(pergunta:str):
    trechos = dados_recuperados.invoke(pergunta)
    contexto = "\n\n".join(um_trecho.page_content for um_trecho in trechos)
    return cadeia.invoke({"query": pergunta, "contexto": contexto})

print(responder("Como devo proceder caso tenha um item comprado roubado e caso eu tenha o cartão gold?"))

Se você tiver um item comprado com o cartão Mastercard Gold e ele for roubado dentro do prazo exigido, é importante informar os dados para processamento do sinistro. Além disso, é recomendável:

1. Entregar o boletim de ocorrência policial referente ao incidente dentro do prazo exigido.
2. Informar as informações exigidas (recibo original de pagamento e anúncio impresso ou non-auction com desconto publicado dentro de 30 dias a partir da data de compra original).
3. Ser colaborativo e fornecer todas as informações adicionais que forem solicitadas para o processamento do sinistro.

Cabendo ao Portador de Cartão tomar as medidas cabíveis para proteger, salvaguardar e/ou recuperar o bem, é importante que você tenha feito todo o possível para recuperar o item.
